## TODO
1. Implement below proof of concept to sec_API.py

## Proof of concept for extracting the financial statement from a report

In [10]:
import os
import pandas as pd
from sec_API import *
from bs4 import BeautifulSoup
import requests
from io import StringIO

In [11]:
edgar = Edgar(os.environ.get('sec-user-agent'))

In [12]:
ticker = 'AMD'
company = edgar.get_company_by_ticker(ticker)

In [13]:
filings = company.filings(form_type='10-Q')
latest_tenQ = filings[0]

In [14]:
balance_sheet = latest_tenQ.reports(report_name='Condensed Consolidated Balance Sheets')[0]

In [15]:
endpoint =  f'https://sec.gov/Archives/edgar/data/{company.cik}/{str(latest_tenQ.accession).replace("-","")}/{balance_sheet.html_filename}'

In [16]:
response = requests.get(
    endpoint,
    headers=edgar.request_headers
).content.decode('utf-8')

balance_sheet = BeautifulSoup(response, 'lxml-xml')

In [17]:
html = str(balance_sheet.find('table', class_ = 'report'))

In [18]:
df = pd.read_html(StringIO(html))[0]
df.set_index(df.columns[0], inplace=True)
df

,"Jun. 29, 2024","Dec. 30, 2023"
"Condensed Consolidated Balance Sheets - USD ($) shares in Millions, $ in Millions",,
Current assets:,NaN,NaN
Cash and cash equivalents,"$ 4,113","$ 3,933"
Short-term investments,1227,1840
"Accounts receivable, net",5749,5376
Inventories,4991,4351
Receivables from related parties,24,9
Prepaid expenses and other current assets,1361,1259
Total current assets,17465,16768
"Property and equipment, net",1666,1589
